# Gebietsstammdaten ZH – Python Demo

This notebook demonstrates the Python port of the R scripts:
- `api/Beispielabfragen.R`
- `script_vorlagen/Gemeinden_ZH_HIST.R`

By default it avoids external network calls. Set `ALLOW_NETWORK = True` to fetch real data.

In [7]:
import sys
from pathlib import Path

repo_root = Path.cwd().resolve().parent
src_path = repo_root / "src"
if src_path.exists():
    sys.path.insert(0, str(src_path))

from gebietsstammdaten import (
    api_health,
    bezirk_mit_gemeinden,
    gemeinde_suchen,
    read_csv_from_url,
    build_gemeinde_historisierung,
    write_csv_rows,
)

ALLOW_NETWORK = True

## API examples (dry-run by default)

In [9]:
# Dry-run shows the request that would be made
api_health(dry_run=False)

{'status': ['healthy'],
 'timestamp': ['2026-02-08 16:14:29'],
 'data_available': [True]}

In [10]:
gemeinde_suchen("Bülach", dry_run=False)

[{'gemeinde_code': [53],
  'gemeinde_name': ['Bülach'],
  'bezirk': [{'bezirk_code': 103, 'bezirk_name': 'Bülach'}]}]

In [11]:
bezirk_mit_gemeinden(101, dry_run=False)

{'bezirk': [{'bezirk_code': 101, 'bezirk_name': 'Affoltern'}],
 'gemeinden': [{'gemeinde_code': 1, 'gemeinde_name': 'Aeugst am Albis'},
  {'gemeinde_code': 2, 'gemeinde_name': 'Affoltern am Albis'},
  {'gemeinde_code': 3, 'gemeinde_name': 'Bonstetten'},
  {'gemeinde_code': 4, 'gemeinde_name': 'Hausen am Albis'},
  {'gemeinde_code': 5, 'gemeinde_name': 'Hedingen'},
  {'gemeinde_code': 6, 'gemeinde_name': 'Kappel am Albis'},
  {'gemeinde_code': 7, 'gemeinde_name': 'Knonau'},
  {'gemeinde_code': 8, 'gemeinde_name': 'Maschwanden'},
  {'gemeinde_code': 9, 'gemeinde_name': 'Mettmenstetten'},
  {'gemeinde_code': 10, 'gemeinde_name': 'Obfelden'},
  {'gemeinde_code': 11, 'gemeinde_name': 'Ottenbach'},
  {'gemeinde_code': 12, 'gemeinde_name': 'Rifferswil'},
  {'gemeinde_code': 13, 'gemeinde_name': 'Stallikon'},
  {'gemeinde_code': 14, 'gemeinde_name': 'Wettswil am Albis'}]}

## Historisierung example with inline sample data

In [5]:
sample_current = [
    {"gemeinde_code": "1", "gemeinde_name": "Neuhausen"},
]

sample_mutations = [
    {
        "mutationsdatum": "2024-01-01",
        "gemeinde_code_neu": "1",
        "gemeinde_code_alt": "10",
        "gemeinde_name_alt": "Alt-Neuhausen",
    },
]

rows = build_gemeinde_historisierung(
    sample_current,
    sample_mutations,
    start_year=2023,
    end_year=2024,
)
rows

[{'gemeinde_code': '1', 'gemeinde_name': 'Neuhausen', 'jahr': 2024},
 {'gemeinde_code': '10', 'gemeinde_name': 'Alt-Neuhausen', 'jahr': 2023}]

## Optional: fetch real OGD data and build CSV

In [6]:
if ALLOW_NETWORK:
    gemeindemutationen_url = (
        "https://www.web.statistik.zh.ch/ogd/daten/ressourcen/"
        "KTZH_00003082_00006504.csv"
    )
    gemeinden_aktuell_url = (
        "https://www.web.statistik.zh.ch/ogd/daten/ressourcen/"
        "KTZH_00003082_00006503.csv"
    )

    gemeindmutationen = read_csv_from_url(gemeindemutationen_url)
    gemeinden_aktuell = read_csv_from_url(gemeinden_aktuell_url)

    rows = build_gemeinde_historisierung(gemeinden_aktuell, gemeindmutationen)
    write_csv_rows(rows, Path("daten/gemeinde_code_jahr.csv"))
    print(rows[:5])

[{'gebietstyp_code': '3', 'gemeinde_code': '1', 'gemeinde_name': 'Aeugst am Albis', 'jahr': 2026}, {'gebietstyp_code': '3', 'gemeinde_code': '10', 'gemeinde_name': 'Obfelden', 'jahr': 2026}, {'gebietstyp_code': '3', 'gemeinde_code': '100', 'gemeinde_name': 'Stadel', 'jahr': 2026}, {'gebietstyp_code': '3', 'gemeinde_code': '101', 'gemeinde_name': 'Steinmaur', 'jahr': 2026}, {'gebietstyp_code': '3', 'gemeinde_code': '102', 'gemeinde_name': 'Weiach', 'jahr': 2026}]
